integrated_test_00
--------------------

Check ray generation for plasma sources. Inparticular we want to check that emissivity to ray calculations are being performed correctly including solid angle and bundle volume calculations.

In [1]:
# Only needed for development in cases where xicsrt is not installed.
import sys
sys.path.append('/u/npablant/code/mirproject/xicsrt')

In [2]:
%matplotlib notebook
import numpy as np

## Initialize the logging module, and exclude messages from included libraries.
import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('PIL').setLevel(logging.WARNING)

import xicsrt

In [13]:
# 1.
config = dict()

# 2.
config['general'] = {}
config['general']['number_of_iter'] = 1
config['general']['number_of_runs'] = 1

# 3.
config['sources'] = {}
config['sources']['source'] = {}
config['sources']['source']['class_name'] = 'XicsrtPlasmaCubic'
config['sources']['source']['origin'] = [0.0, 0.0, 0.0]
config['sources']['source']['xsize'] = 0.01
config['sources']['source']['ysize'] = 0.01
config['sources']['source']['zsize'] = 0.01
config['sources']['source']['target'] = [0.0, 0.0, 1.0]

config['sources']['source']['emissivity'] = 1e12
config['sources']['source']['time_resolution'] = 1
config['sources']['source']['spread'] = np.radians(180)


# 4.
config['optics'] = {}
config['optics']['detector'] = {}
config['optics']['detector']['class_name'] = 'XicsrtOpticDetector'
config['optics']['detector']['origin'] = [0.0,  0.0, 1.0]
config['optics']['detector']['zaxis']  = [0.0, 0.0, -1.0]
config['optics']['detector']['xsize']  = 0.1
config['optics']['detector']['ysize']  = 0.1
config['optics']['detector']['check_size'] = False

# 6.
results = xicsrt.raytrace(config)

DEBUG:root:The xicrsrt_contrib package is not installed.
INFO:root:Starting run: 1 of 1
DEBUG:root:The xicrsrt_contrib package is not installed.
INFO:root:Seeding np.random with None
DEBUG:root:Creating filters
DEBUG:root:Creating sources
DEBUG:root:Creating optics
DEBUG:XicsrtOpticDetector:Pixel grid size: 100 x 100
INFO:root:Starting iteration: 1 of 1
DEBUG:root:Generating rays
DEBUG:XicsrtPlasmaCubic:Starting setup_bundles
DEBUG:XicsrtPlasmaCubic:Starting bundle_filter
DEBUG:XicsrtPlasmaCubic:Starting bundle_generate
DEBUG:XicsrtPlasmaCubic:Predicted rays: 1.00e+06
DEBUG:root:Bundles Generated:       1.0000e+00
DEBUG:root:Rays per bundle, mean:   1000000
DEBUG:root:Rays per bundle, median: 1000000
DEBUG:root:Rays per bundle, max:    1000000
DEBUG:root:Rays per bundle, min:    1000000
DEBUG:root:Raytracing optics
DEBUG:XicsrtOpticDetector: Rays on   XicsrtOpticDetector:   5.0017e+05
DEBUG:XicsrtOpticDetector: Rays from XicsrtOpticDetector: 5.0017e+05



Rays Generated: 1.000e+06
Rays Detected:  5.002e+05
Efficiency:     5.002e-01 ± 7.1e-04 (50.01680%)



In [15]:
num_expected = (config['sources']['source']['emissivity']
                * config['sources']['source']['xsize']
                * config['sources']['source']['ysize']
                * config['sources']['source']['zsize'])
num_actual = results['total']['meta']['source']['num_out']
print(f'Rays generated at source')
print(f'  num_expected: {num_expected} num_actual: {num_actual}')
np.testing.assert_allclose(num_expected, num_actual, 1)

num_exp_detector = num_expected/2
num_act_detector = results['total']['meta']['detector']['num_out']
print(f'Rays on detector plane')
print(f'  num_expected: {num_exp_detector} num_actual: {num_act_detector}')
sigma_5 = np.sqrt(num_exp_detector)*5
np.testing.assert_allclose(num_exp_detector, num_act_detector, sigma_5)

Rays generated at source
  num_expected: 1000000.0 num_actual: 1000000
Rays on detector plane
  num_expected: 500000.0 num_actual: 500168
